# Investigating The Efficacy of Data Profiles in the Prediction of Pulsar Star Class.

## Introduction

A pulsar star is a neutron star with a high mass that emits electromagnetic radiation when it collapses. This emission of radiation can be detected as radio waves by telescopes thus giving them a unique integrated pulse profile and DM-SNR curve. These profiles can be used to classify a signal as either pulsar or non-pulsar.

### Main Question

Does the integrated pulse profile or the DM-SNR curve better predict whether an emission signal is from a pulsar star? 

### The Dataset

This dataset (https://archive.ics.uci.edu/ml/datasets/HTRU2) represents a collection of observed radio emissions that have been confirmed to be either from pulsar stars (Class 1), or a mix of radio frequency interference (RFI) and noise (Class 0).  
Each observation contains a mean, standard deviation, kurtosis, and skewness of emissions averaged over multiple rotations of the potential pulsar star. This makes up the integrated profile (IP).  
Each observation also contains a transformed mean, standard deviation, kurtosis, and skewness of the emissions based on the dispersion measure (DM) to maximize the signal-to-noise ratio. These make up the DM-SNR variables.


In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
install.packages("themis")
install.packages("expss")
library(expss)
source('tests.R')
source('cleanup.R')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

ERROR: Error in library(expss): there is no package called ‘expss’


In [4]:
#Reading the data: Here, we read the dataset from the web into R, 
# and loaded the dataset to examine it.
options(repr.matrix.max.rows = 6)
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00372/HTRU2.zip"
download.file(url, "HTRU2.zip") 
unzip("HTRU2.zip")
pulsar_data_raw <- read_csv('HTRU_2.csv') 
pulsar_data_raw


Rows: 17897 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): 140.5625, 55.68378214, -0.234571412, -0.699648398, 3.199832776, 19....

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


140.5625,55.68378214,-0.234571412,-0.699648398,3.199832776,19.11042633,7.975531794,74.24222492,0
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
102.5078,58.88243,0.46531815,-0.5150879,1.677258,14.86015,10.576487,127.39358,0
103.0156,39.34165,0.32332837,1.0511644,3.121237,21.74467,7.735822,63.17191,0
136.7500,57.17845,-0.06841464,-0.6362384,3.642977,20.95928,6.896499,53.59366,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
119.3359,59.93594,0.1593631,-0.74302540,21.430602,58.87200,2.499517,4.595173,0
114.5078,53.90240,0.2011614,-0.02478884,1.946488,13.38173,10.007967,134.238910,0
57.0625,85.79734,1.4063910,0.08951971,188.306020,64.71256,-1.597527,1.429475,0


## Tidying and preparing the Data:

In [6]:
# Here, we tidied and wrangled the data set using these steps: 
# a) Giving names to columns 
# b) Changing the Class column to a factor 
# c) Splitting the data into the IP and DM-SNR profiles
# d) Spitting each profile into a training and test set

options(repr.matrix.max.rows = 6)
col_names <- c('Mean_IP', 'Stdev_IP', 'Kurtosis_IP', 'Skewness_IP', 
               'Mean_DM', "Stdev_DM", 'Kurtosis_DM', "Skewness_DM", "Class")
pulsar_data <- read_csv('HTRU_2.csv', col_names)

pulsar_data_wrangled <- mutate(pulsar_data, Class = as.factor(Class))

pulsar_data_IP <- select(pulsar_data_wrangled, Mean_IP, Stdev_IP, Kurtosis_IP, Skewness_IP, Class)

IP_split <- initial_split(pulsar_data_IP, prop = 0.75, strata = Class)
IP_train <- training(IP_split)
IP_test <- testing(IP_split) 

IP_train

pulsar_data_DM <- select(pulsar_data_wrangled, Mean_DM, Stdev_DM, Kurtosis_DM, Skewness_DM, Class)

DM_split <- initial_split(pulsar_data_DM, prop = 0.75, strata = Class)
DM_train <- training(DM_split)
DM_test <- testing(DM_split) 

DM_train


Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): Mean_IP, Stdev_IP, Kurtosis_IP, Skewness_IP, Mean_DM, Stdev_DM, Kur...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Mean_IP,Stdev_IP,Kurtosis_IP,Skewness_IP,Class
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
102.5078,58.88243,0.46531815,-0.5150879,0
103.0156,39.34165,0.32332837,1.0511644,0
136.7500,57.17845,-0.06841464,-0.6362384,0
⋮,⋮,⋮,⋮,⋮
119.3359,59.93594,0.1593631,-0.74302540,0
114.5078,53.90240,0.2011614,-0.02478884,0
57.0625,85.79734,1.4063910,0.08951971,0


Mean_DM,Stdev_DM,Kurtosis_DM,Skewness_DM,Class
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
3.199833,19.11043,7.975532,74.24222,0
1.677258,14.86015,10.576487,127.39358,0
3.642977,20.95928,6.896499,53.59366,0
⋮,⋮,⋮,⋮,⋮
1.296823,12.16606,15.450260,285.931022,0
21.430602,58.87200,2.499517,4.595173,0
1.946488,13.38173,10.007967,134.238910,0


## Summary Table of Each Dataset:

In [ ]:
#We created a table of the means, standard deviations, kurtosis, 
# and skewness of the integrated and the DM-SNR puslar profiles.

pulsar_data_summary <- group_by(pulsar_data, Class) |>
    summarize(avg_mean_ip = mean(Mean_IP), mean_stdev_ip = mean(Stdev_IP), 
              avg_mean_dm = mean(Mean_DM), mean_stdev_dm = mean(Stdev_DM), 
              count = n(),
             max_ip = max(Mean_IP), min_ip = min(Mean_IP)) |>
    mutate (as.factor(Class)) 

IP_train_summary <- group_by(IP_train, Class) |>
    summarize(avg_mean_ip = mean(Mean_IP), avg_stdev_ip = mean(Stdev_IP), 
              avg_kurtosis_ip = mean(Kurtosis_IP), avg_skewness_ip = mean(Skewness_IP),
              count = n(), percentage = n() / IP_num_obs * 100) |>
    table_name("Integrated profile training data summary")
IP_train_summary

DM_train_summary <- group_by(DM_train, Class) |>
    summarize(avg_mean_dm = mean(Mean_DM), avg_stdev_dm = mean(Stdev_DM), 
              avg_kurtosis_dm = mean(Kurtosis_DM), avg_skewness_dm = mean(Skewness_DM),
              count = n(), percentage = n() / DM_num_obs * 100)
DM_train_summary

## Visualizing the data

In [ ]:
#We visualized the distributions of means, SD, 
# kurtoses and skewness for the integrated and DM-SNR profiles.
pulsar_hist_mean_IP <- ggplot(IP_train, aes(x = Mean_IP, fill = Class)) +
    geom_histogram(alpha = 0.7, position = "identity", bins = 30) +
    labs(x="Mean of integrated profile", 
         y= "Number of stars", 
         fill = "Pulsar or \nnot Pulsar", 
         title = 'Distribution of means via integrated profile')
pulsar_hist_mean_IP

pulsar_hist_SD_IP <- ggplot(IP_train, aes(x = Stdev_IP, fill = Class)) +
    geom_histogram(alpha = 0.7, position = "identity", bins = 30) +
    labs(x="Standard deviation of integrated profile", 
         y= "Number of stars", 
         fill = "Pulsar or \nnot Pulsar", 
         title = 'Distribution of standard deviations via integrated profile')
pulsar_hist_SD_IP

pulsar_hist_Kurtosis_IP <- ggplot(IP_train, aes(x = Kurtosis_IP, fill = Class)) +
    geom_histogram(alpha = 0.7, position = "identity", bins = 30) +
    labs(x="Kurtosis of integrated profile", 
         y= "Number of stars", 
         fill = "Pulsar or \nnot Pulsar", 
         title = 'Distribution of kurtoses via integrated profile')
pulsar_hist_Kurtosis_IP

pulsar_hist_Skew_IP <- ggplot(IP_train, aes(x = Skewness_IP, fill = Class)) +
    geom_histogram(alpha = 0.7, position = "identity", bins = 30) +
    labs(x="Skewness of integrated profile", 
         y= "Number of stars", 
         fill = "Pulsar or \nnot Pulsar", 
         title = 'Distribution of skew via integrated profile')
pulsar_hist_Skew_IP


pulsar_hist_mean_DM <- ggplot(DM_train, aes(x = Mean_DM, fill = Class)) +
    geom_histogram(alpha = 0.7, position = "identity", bins = 30) +
    labs(x="Mean of DM-SNR", 
         y= "Number of stars", 
         fill = "Pulsar or \nnot Pulsar", 
         title = 'Distribution of means via DM-SNR')
pulsar_hist_mean_DM

pulsar_hist_SD_DM <- ggplot(DM_train, aes(x = Stdev_DM, fill = Class)) +
    geom_histogram(alpha = 0.7, position = "identity", bins = 30) +
    labs(x="Standard deviation of DM-SNR", 
         y= "Number of stars", 
         fill = "Pulsar or \nnot Pulsar", 
         title = 'Distribution of standard deviations via DM-SNR')
pulsar_hist_SD_DM

pulsar_hist_Kurtosis_DM <- ggplot(DM_train, aes(x = Kurtosis_DM, fill = Class)) +
    geom_histogram(alpha = 0.7, position = "identity", bins = 30) +
    labs(x="Kutosis of DM-SNR", 
         y= "Number of stars", 
         fill = "Pulsar or \nnot Pulsar", 
         title = 'Distribution of kurtoses via DM-SNR')
pulsar_hist_Kurtosis_DM

pulsar_hist_Skew_DM <- ggplot(DM_train, aes(x = Skewness_DM, fill = Class)) +
    geom_histogram(alpha = 0.7, position = "identity", bins = 30) +
    labs(x="Skewness of DM-SNR", 
         y= "Number of stars", 
         fill = "Pulsar or \nnot Pulsar", 
         title = 'Distribution of skew via DM-SNR')
pulsar_hist_Skew_DM

### Normal distributions of average means of each profile

We wanted to see if there is a greater difference between positive and negative pulsar stars in either profile so that we could hypothesize which one would more accurately predict class of a pulsar star. However, it is very difficult to see a difference with the naked eye.

In [ ]:
#We visualized the curve of the average mean of the integrated pulse profile, 
# and the DM-SNR profile for positive and negative pulsar stars.
IP_distribution <- ggplot() + 
  lapply(split(pulsar_data_IP_summary, seq_len(nrow(pulsar_data_IP_summary))), 
         \(x) stat_function(fun = dnorm, 
                            args = list(mean = x$avg_mean_ip, sd = x$mean_stdev_ip), 
                            aes(color = factor(x$Class)))) +
  xlim(-100,300)+
    labs(x = 'Mean via IP', y = 'Density', color = 'Class', 
         title = 'Distribution of means using IP')
IP_distribution

DM_distribution <- ggplot() + 
  lapply(split(pulsar_data_DM_summary, seq_len(nrow(pulsar_data_DM_summary))), 
         \(x) stat_function(fun = dnorm, 
                            args = list(mean = x$avg_mean_dm, sd = x$mean_stdev_dm), 
                            aes(color = factor(x$Class)))) +
  xlim(-100,300)+
    labs(x = 'Mean via DM-SNR', y = 'Density', color = 'Class', 
         title = 'Distribution of means using DM-SNR')
DM_distribution

## Methods

1. Creating 2 data groups:
   - Group 1: Integrated profile columns Mean_IP, Stdev_IP, Kurtosis_IP and Skewness_IP will be used to predict the Class
   - Group 2: DM-SNR profile columns Mean_DM, Stdev_DM, Kurtosis_DM and Skewness_DM will be used to predict the Class
2. Oversampling: When running knn we will use the oversampling technique since positive cases are very few compared to negative cases ((1,639 positive examples (9.16%) and 16,259 negative examples (90.85%)).
3. We will also standardize our variables.
4. Training the classification KNN models
5. Tuning the models
6. Testing the models 
7. Evaluating the models via confusion matrices and accuracy estimates
8. Picking the best classification model to predict pulsar stars based on background information.
For example, if it is more costly to investigate a false positive, we would want to choose a model with high accuracy and a low rate of false positives.



### Visualizing the results

We will visualize the results with a bar graph comparing the accuracies of each prediction model we created. 

## Conclusion: 

To make a hypothesis on which profile is better at predicting pulsar star class, we computed if the difference in means between pulsar and non-pulsar stars is significant for both profiles. Although both profiles are effective at telling a pulsar and non-pulsar star apart (p < 0.05), the integrated profile has a larger difference between the mean of pulsar and non-pulsar (larger t).

### Hypothesis: 
We hypothesize that using the integrated profile versus the DM-SNR profile will provide more accurate predictions of pulsar star class.


In [ ]:
set.seed(0)

integrated_pulsar <- rnorm(1639, mean = 56.69061, sd = 38.71060)
integrated_non_pulsar <- rnorm(16259, mean= 116.56273, sd = 47.33974)

integrated_profile_t_test_result <- t.test(integrated_non_pulsar, integrated_pulsar, alternative = "two.sided", var.equal = TRUE)
integrated_profile_t_test_result

DM_pulsar <- rnorm(1639, mean = 8.863258, sd = 23.28798)
DM_non_pulsar <- rnorm(16259, mean= 49.825995, sd = 56.46896)

DM_profile_t_test_result <- t.test(DM_non_pulsar, DM_pulsar, alternative = "two.sided", var.equal = TRUE)
DM_profile_t_test_result

Investigating which profile best predicts the class of a pulsar star can help us to better understand and identify these cosmic phenomena. Thus, this leads us to future questions such as: if a pulsar star has been identified, what is the correlation of its findings to other cosmic phenomena such as dark matter and gravity waves? 

# knn classification

In [ ]:
#upsampling pulsar stars
library (themis)

ups_recipe <- recipe(Class ~ ., data = pulsar_data_IP) |>
  step_upsample(Class, over_ratio = 1, skip = FALSE) |>
  prep()

ups_recipe

In [ ]:
set.seed(17)

IP_split <- initial_split(pulsar_data_IP, prop = 0.75, strata = Class)
IP_train <- training(IP_split)
IP_test <- testing(IP_split) 

IP_proportions <- IP_train |>
                      group_by(Class) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(IP_train))

IP_proportions

IP_recipe <- recipe(Class ~ ., data = IP_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) 

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(IP_recipe) |>
  add_model(knn_spec) |>
  fit(data = IP_train)

IP_predictions <- predict(knn_fit, IP_test) |>
  bind_cols(IP_test) |>
  conf_mat(truth = Class, estimate = .pred_class)
IP_predictions



In [ ]:
#knn with cross-validation to pick K
IP_split <- initial_split(pulsar_data_IP, prop = 0.75, strata = Class)
IP_train <- training(IP_split)
IP_test <- testing(IP_split) 

IP_recipe <- recipe(Class ~ ., data = IP_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors()) 

IP_vfold <- vfold_cv(IP_train, v = 5, strata = Class)

IP_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_fit <- workflow() |>
  add_recipe(IP_recipe) |>
  add_model(IP_spec) |>
  fit_resamples(resamples = IP_vfold) |>
  collect_metrics()

validation_predicted <- predict(knn_fit, cancer_validation) |>
  bind_cols(cancer_validation)

k_vals <- tibble(neighbors = seq(from = 1, to = 100, by = 5))

IP_results <- workflow() |>
  add_recipe(IP_recipe) |>
  add_model(IP_spec) |>
  tune_grid(resamples = IP_vfold, grid = k_vals) |>
  collect_metrics() 

accuracies <- IP_results |>
  filter(.metric == "accuracy")
accuracies